# Model Setup
Model 1 takes in img outputs moving probability
Model 2 takes in a sequence of 16 moving probs outputs falling probability

In [2]:
import os

import numpy as np
import datetime
import matplotlib.pyplot as plt

import tensorflow as tf
assert tf.__version__.startswith('2')
import tensorflow_datasets as tfds

from tflite_model_maker import model_spec
from tflite_model_maker import image_classifier
from tflite_model_maker.config import ExportFormat
from tflite_model_maker.config import QuantizationConfig

from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras import layers


import cv2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, RandomFlip, RandomRotation
from tensorflow.keras.utils import Sequence

 

/home/kevin/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/kevin/.local/lib/python3.8/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
/home/kevin/.local/lib/python3.8/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.10.0 and strictly below

GPU limitation

In [3]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    # Restrict TensorFlow to only allocate 2GB of memory on the first GPU
    try:
        tf.config.set_logical_device_configuration(gpus[0], [tf.config.LogicalDeviceConfiguration(memory_limit=2048)])
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Virtual devices must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


2023-08-07 16:01:18.452846: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-08-07 16:01:18.572300: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-08-07 16:01:18.572557: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-08-07 16:01:18.573591: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

# Building the Models

## Optimized Model 1

In [4]:
#cut a video into different frames
def cut_video(filename):
    list_of_frames = []
    cap = cv2.VideoCapture(filename)
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_count  = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        list_of_frames.append(frame)
   
    cap.release()
    return np.array(list_of_frames)


# loop through each frame in the array of images, take two frames at a time, 
# # loop through each pixel in the 2 frames, find the differences in values 
# in grey scale for the corresponding pixels, append the diff to the rgb array 
# representing each pixel =>>>>>>>> RGBX. Notei if first frame append 0 to rgb array
def load_images_from_folders(data_path,total): 
    images = np.zeros((total,224,224,4),dtype=int)
    counter = 0

    #loop through each vid in the folder 
    for index in range(len(data_path)): #looping through the entire folder array
        
        #variable to track whether it is the first image/frame in the video
        first_image = True

        #cut the video into different frames and save them all into an array
        array_images = cut_video(data_path[index]) # array_images[0][0][0] = single pixel

        #array to store modified pixel of each image, later on append it to images[]
        
  
        for i in range(len(array_images)-1):
            if first_image == True:
                #process for 0th and 1st frame
                images[counter] = first_frame_pixel_modification(array_images[i])
                first_image = False
                counter+=1
            
            images[counter] = two_frame_pixel_modification(array_images[i],array_images[i+1])
            counter+=1
        
    #         print("finished frame " + str(i)+ " of video " + str(index))
        
    #     print(str(counter) +" matches " + str(len(array_images)))
    #     print("finished video " + str(index))
        
    #     #after finish loop through a frame, append the frame into the images[] array
    # print("final count: " + str(counter) + "; expected count = " + str(total))
    print(counter)
    # print(images.shape)
    return images

def two_frame_pixel_modification(frame1, frame2): 
    row_frame = np.zeros((224,224,4),dtype=int)
    np.array(row_frame) 
    for r in range(len(frame1)):
        column_frame = np.zeros((224,4),dtype=int)
        for c in range(len(frame1[0])):
            #calcualte each pixel's difference 
            grey1 =  find_greyscale(frame1[r][c][2],frame1[r][c][1],frame1[r][c][0])
            grey2 =  find_greyscale(frame2[r][c][2],frame2[r][c][1],frame2[r][c][0])
            diff_grey = abs(grey1-grey2)
            new_rgbx = [frame2[r][c][2],frame2[r][c][1],frame2[r][c][0],diff_grey] #add greyscale diff to the end of the pixel array, and append it to the individual frame array
            new_rgbx = np.array(new_rgbx)
            column_frame[c] = np.array(new_rgbx)
        
        row_frame[r] = column_frame
 
        
    return row_frame 

def first_frame_pixel_modification(frame0):
    row_frame = np.zeros((224,224,4),dtype=int)
    np.array(row_frame)
    for r in range(len(frame0)):

        column_frame = np.zeros((224,4),dtype=int)
        for c in range(len(frame0[0])): 

            new_rgbx = [frame0[r][c][2],frame0[r][c][1],frame0[r][c][0],0]
            column_frame[c] = np.array(new_rgbx)

        row_frame[r] = column_frame

    return row_frame
    

def find_greyscale(r,g,b):
    grey = 0.299*r + 0.587*g + 0.114*b
    return grey

def cal_totalframes(array_of_vids):
    total_frames = 0
    for i in range(len(array_of_vids)):
        list_of_cut_frames = cut_video(array_of_vids[i])
        total_frames+= len(list_of_cut_frames)
    return total_frames




moving_path = [
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall2.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall3.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall4.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall1.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall5.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall6.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall7.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall8.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall9.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall10.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall11.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall12.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall13.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall14.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall15.mp4"
                 ]

still_path = [
                 "./../datasets/vids/splitted/new_still/resized_logitech-default1.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default2.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default3.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default4.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default5.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default6.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default7.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default8.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default9.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default10.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default11.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default12.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default13.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default14.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default15.mp4"
                 ]

still_data = load_images_from_folders(still_path,cal_totalframes(still_path))
moving_data = load_images_from_folders(moving_path,cal_totalframes(moving_path))



567
513


### Prepare data

In [8]:
#concatenate still and movind data to create inputs and outputs
inputs = np.concatenate((still_data, moving_data))

outputs = np.concatenate((np.zeros(len(still_data)),np.ones(len(moving_data))))


#shuffle the data to prepare for training 
shuffled_indices = np.random.permutation(len(inputs))
inputs, outputs = inputs[shuffled_indices],outputs[shuffled_indices]

inputs = np.concatenate((inputs,inputs))
outputs = np.concatenate((outputs,outputs))

inputs = inputs.astype(np.float32)
outputs = outputs.astype(np.float32)
print(inputs)

#create a tensorflow dataset
with tf.device('/cpu:0'):
    dataset = tf.data.Dataset.from_tensor_slices((inputs,outputs))
    dataset = dataset.batch(16).prefetch(buffer_size=tf.data.AUTOTUNE)



#determine train and test 
train_data = dataset.take(int(0.8*len(dataset)))
val_data = dataset.skip(int(0.2*len(dataset)))






[[[[153. 153. 153.   0.]
   [153. 153. 153.   0.]
   [153. 153. 153.   0.]
   ...
   [237. 253. 255.   0.]
   [237. 253. 255.   0.]
   [237. 253. 255.   0.]]

  [[154. 154. 154.   0.]
   [154. 154. 154.   0.]
   [154. 154. 154.   0.]
   ...
   [237. 253. 255.   0.]
   [237. 253. 255.   0.]
   [237. 253. 255.   0.]]

  [[154. 154. 154.   0.]
   [154. 154. 154.   0.]
   [154. 154. 154.   0.]
   ...
   [238. 255. 255.   1.]
   [238. 255. 255.   1.]
   [238. 255. 255.   1.]]

  ...

  [[  0.   3.   9.   1.]
   [  0.   0.   1.   1.]
   [ 33.  41.  50.  35.]
   ...
   [ 15.  28.  43.  16.]
   [ 11.  24.  39.  12.]
   [ 11.  24.  39.  12.]]

  [[  0.   0.   8.   1.]
   [ 12.  17.  25.  12.]
   [ 82.  89. 101.  87.]
   ...
   [ 49.  71.  85.  58.]
   [ 40.  62.  76.  49.]
   [ 37.  59.  73.  46.]]

  [[  0.   0.   6.   2.]
   [ 21.  26.  34.  25.]
   [ 94. 101. 113.  90.]
   ...
   [ 53.  75.  89.  62.]
   [ 41.  63.  77.  50.]
   [ 39.  61.  75.  48.]]]


 [[[ 48.  47.  52.  31.]
   [ 50.  49

In [48]:
print(inputs.shape)
print(outputs.shape)

(2160, 224, 224, 4)
(2160,)


### Define Model

In [14]:
input_shape = (2160,224, 224, 4)

#data augmentation 
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.3),
    layers.RandomContrast(factor=0.1)

])


model = tf.keras.Sequential([

    # Add Convolutional and Pooling layers
    data_augmentation,
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    # Flatten the output of the Convolutional layers before passing to Dense layers
    Flatten(),
    
    # Add Dense layers for classification
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid'),  # Using sigmoid activation for binary classification
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# Print the model summary to verify the input shape
model.build(input_shape)
model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_8 (Sequential)   (2160, 224, 224, 4)       0         
                                                                 
 conv2d_12 (Conv2D)          (2160, 222, 222, 32)      1184      
                                                                 
 max_pooling2d_12 (MaxPoolin  (2160, 111, 111, 32)     0         
 g2D)                                                            
                                                                 
 conv2d_13 (Conv2D)          (2160, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_13 (MaxPoolin  (2160, 54, 54, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_14 (Conv2D)          (2160, 52, 52, 128)      

### Train Model

In [15]:

model.fit(train_data,epochs=5,validation_data=val_data)


Epoch 1/5
108/108 [==============================] - 7s 55ms/step - loss: 17.8412 - accuracy: 0.8131 - val_loss: 0.2564 - val_accuracy: 0.9074
Epoch 2/5
108/108 [==============================] - 6s 54ms/step - loss: 0.5411 - accuracy: 0.8160 - val_loss: 0.5405 - val_accuracy: 0.8108
Epoch 3/5
108/108 [==============================] - 5s 51ms/step - loss: 0.4648 - accuracy: 0.8559 - val_loss: 0.2843 - val_accuracy: 0.8912
Epoch 4/5
108/108 [==============================] - 6s 54ms/step - loss: 0.2904 - accuracy: 0.8993 - val_loss: 0.2188 - val_accuracy: 0.9091
Epoch 5/5
108/108 [==============================] - 6s 54ms/step - loss: 0.2045 - accuracy: 0.9306 - val_loss: 0.1282 - val_accuracy: 0.9572


### Evaluate & Export

In [17]:
###TESTING 
test_moving = [ "./../datasets/vids/testdata/moving/resized-test-fall1.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall2.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall3.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall4.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall5.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall6.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall7.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall8.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall9.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall10.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall11.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall12.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall13.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall14.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall15.mp4"
]

test_still = [
                "./../datasets/vids/testdata/still/resized-test-default1.mp4",
                "./../datasets/vids/testdata/still/resized-test-default2.mp4",
                "./../datasets/vids/testdata/still/resized-test-default3.mp4",
                "./../datasets/vids/testdata/still/resized-test-default4.mp4",
                "./../datasets/vids/testdata/still/resized-test-default5.mp4",
                "./../datasets/vids/testdata/still/resized-test-default6.mp4",
                "./../datasets/vids/testdata/still/resized-test-default7.mp4",
                "./../datasets/vids/testdata/still/resized-test-default8.mp4",
                "./../datasets/vids/testdata/still/resized-test-default9.mp4",
                "./../datasets/vids/testdata/still/resized-test-default10.mp4",
                "./../datasets/vids/testdata/still/resized-test-default11.mp4",
                "./../datasets/vids/testdata/still/resized-test-default12.mp4",
                "./../datasets/vids/testdata/still/resized-test-default13.mp4",
                "./../datasets/vids/testdata/still/resized-test-default14.mp4",
                "./../datasets/vids/testdata/still/resized-test-default15.mp4"]

still_tdata = load_images_from_folders(test_still,cal_totalframes(test_still))
moving_tdata = load_images_from_folders(test_moving,cal_totalframes(test_moving))




501
514


In [18]:
#concatenate still and movind data to create inputs and outputs
tinputs = np.concatenate((still_tdata, moving_tdata))
toutputs = np.concatenate((np.zeros(len(still_tdata)),np.ones(len(moving_tdata))))

tinputs = tinputs.astype(np.float32)
toutputs = toutputs.astype(np.float32)

#shuffle the data to prepare for training 
tshuffled_indices = np.random.permutation(len(tinputs))
tinputs, toutputs = tinputs[tshuffled_indices],toutputs[tshuffled_indices]

print(tinputs)
print(toutputs)

#create a tensorflow dataset
with tf.device('/cpu:0'):
    tdataset = tf.data.Dataset.from_tensor_slices((tinputs,toutputs))
    tdataset = tdataset.batch(16).prefetch(buffer_size=tf.data.AUTOTUNE)


# model.summary()

[[[[223. 221. 217.  13.]
   [223. 221. 217.  14.]
   [223. 221. 217.  14.]
   ...
   [223. 223. 218.  14.]
   [223. 223. 218.  14.]
   [223. 223. 218.  14.]]

  [[223. 221. 217.  13.]
   [223. 221. 217.  14.]
   [223. 221. 217.  14.]
   ...
   [223. 223. 218.  14.]
   [223. 223. 218.  14.]
   [223. 223. 218.  14.]]

  [[223. 221. 217.  12.]
   [223. 221. 217.  13.]
   [223. 221. 217.  13.]
   ...
   [223. 223. 218.  14.]
   [223. 223. 218.  14.]
   [223. 223. 218.  14.]]

  ...

  [[129. 139. 145.   9.]
   [129. 139. 145.   9.]
   [129. 139. 145.  10.]
   ...
   [128. 117. 124.   8.]
   [123. 121. 133.   9.]
   [124. 122. 134.   8.]]

  [[129. 139. 145.   9.]
   [129. 139. 145.   9.]
   [129. 139. 145.   9.]
   ...
   [128. 117. 124.  10.]
   [123. 121. 133.  11.]
   [124. 122. 134.  10.]]

  [[129. 139. 145.   9.]
   [129. 139. 145.   9.]
   [129. 139. 145.   9.]
   ...
   [128. 117. 124.  10.]
   [123. 121. 133.  12.]
   [124. 122. 134.  10.]]]


 [[[ 46.  23.  22.   3.]
   [ 46.  23

In [19]:
loss, accuracy = model.evaluate(tdataset)


64/64 [==============================] - 2s 14ms/step - loss: 0.6675 - accuracy: 0.6956


In [20]:
#save model1 
model.save("optimized_model1")

2023-08-07 16:21:16.541109: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: optimized_model1/assets


In [22]:
def representative_dataset():
    for data, label in tf.data.Dataset.from_tensor_slices((inputs, outputs)).batch(1).take(100):
        yield [tf.dtypes.cast(data, tf.float32)]

with tf.device('/cpu:0'):
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    converter.representative_dataset = representative_dataset
    converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
    converter.inference_input_type = tf.uint8
    converter.inference_output_type = tf.uint8
    tflite_quant_m1 = converter.convert()

    # Save the quantized model 1
    with open('optimized_model1.tflite', 'wb') as f: f.write(tflite_quant_m1)

INFO:tensorflow:Assets written to: /tmp/tmp9q22zllm/assets


INFO:tensorflow:Assets written to: /tmp/tmp9q22zllm/assets
/home/kevin/.local/lib/python3.8/site-packages/tensorflow/lite/python/convert.py:746: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "
2023-08-07 16:22:38.201360: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:357] Ignored output_format.
2023-08-07 16:22:38.201386: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:360] Ignored drop_control_dependency.
2023-08-07 16:22:38.201535: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/tmp9q22zllm
2023-08-07 16:22:38.204671: I tensorflow/cc/saved_model/reader.cc:78] Reading meta graph with tags { serve }
2023-08-07 16:22:38.204684: I tensorflow/cc/saved_model/reader.cc:119] Reading SavedModel debug info (if present) from: /tmp/tmp9q22zllm
2023-08-07 16:22:38.216230: I tensorflow/cc/saved_model/loader.cc:2

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

### Batch predicting multiple vids

In [14]:
falling_paths = [
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall2.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall3.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall4.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall1.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall5.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall6.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall7.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall8.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall9.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall10.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall11.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall12.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall13.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall14.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall15.mp4"
                 ]

default_paths = [
                 "./../datasets/vids/splitted/new_still/resized_logitech-default1.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default2.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default3.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default4.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default5.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default6.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default7.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default8.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default9.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default10.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default11.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default12.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default13.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default14.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default15.mp4"
                 ]


test_moving = [


In [16]:
def batchPredict(path):
    cap = cv2.VideoCapture(path)
    probs = []
    while True:
        ret, frame = cap.read() 
        if not ret: break

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame_rgb = np.expand_dims(frame_rgb, axis=0)

        interpreter = tf.lite.Interpreter(model_path="model.tflite")
        interpreter.allocate_tensors()
        output = interpreter.get_output_details()
        input = interpreter.get_input_details()
        output_index = output[0]['index']
        input_index = input[0]['index']

        interpreter.set_tensor(input_index, frame_rgb)
        interpreter.invoke()
        output_data = interpreter.get_tensor(output_index)
        output_data = output_data[0]
        output_probs = tf.nn.softmax(output_data.astype(float))

        predicted_index = np.argmax(output_data)
        class_labels = ["Moving", "Still"]
        predicted_class = class_labels[predicted_index]        
        prob = np.around(max(output_probs.numpy()), decimals = 2)

        if predicted_class == "Still": probs.append(np.subtract(1, prob))
        else: probs.append(prob)
    return probs

In [ ]:
def batchPredictMultVids(vid_path_lst):
    record = []
    for vid in vid_path_lst:
        vid_probs = batchPredict(vid)
        print(vid)
        for i in range(len(vid_probs)-15):
            record.append(vid_probs[i:i+16])
            for j in range(i, i+16):
                print(vid_probs[j], end=" ")
            print()
        print()
    return record

In [15]:
falling = batchPredictMultVids(falling_paths)
default = batchPredictMultVids(default_paths)

NameError: name 'batchPredictMultVids' is not defined

## Model 2

### Train

In [ ]:
default_data = np.loadtxt('./datasets/model2_data/default.txt')
falling_data = np.loadtxt('./datasets/model2_data/falling.txt')

print(falling_data.shape, default_data.shape)

(2344, 16) (6270, 16)


In [ ]:

#augment the falling data

mu, sig = 0, 0.001
copy_count = len(default_data)-len(falling_data)
oldLen = len(falling_data)
rowi = 0

# limit = lambda x : max(min(x,1), 0), basically 0 <= x < 1

def limit(x):
    if x > 1: return 1 - (x-1) # if x larger than 1 -> take the difference between x and 1 and subtract from 1
    elif x < 0: return abs(x) # if x is smaller than 0 -> abs val it 
    return x

newLimit = np.vectorize(limit) #Vectorize to loop through the entire array at once

for i in range(copy_count):
    row = falling_data[rowi] + np.random.normal(mu, sig, 16)
    print(row)
    row = newLimit(row)

    falling_data = np.vstack([falling_data, row])
    rowi += 1
    if rowi >= oldLen: rowi = 0

print(falling_data.shape, default_data.shape)

(6270, 16) (6270, 16)


In [ ]:
# loading data, output is used to validate input
inputs = np.concatenate((default_data, falling_data))
outputs = np.concatenate((np.zeros(len(default_data)), np.ones(len(falling_data)))) #ones are falling , zeros are default


dataset_size = len(inputs) #length of the entire dataset, including both outputs and inputs
new_indices = np.random.permutation(dataset_size) # using dataset size, shufflen the indices to shuffle X and y at the same time
print(new_indices)

#new_indices is an array that contains all indices of the dataset in a shuffled order, when passed into inputs/outputs, other np arrays, inputs and outputs will be rearranged to match the index of the new_indices array
inputs, outputs = inputs[new_indices], outputs[new_indices] 


train_size = int(0.8*dataset_size)
test_size = dataset_size - train_size


# train_data = tf.data.Dataset.from_tensor_slices((inputs[:train_size], outputs[:train_size])).batch(128)
X_train, y_train = inputs[:train_size], outputs[:train_size] #x = images, y = label 
X_test, y_test = inputs[train_size:], outputs[train_size:]


# Define the model architecture
model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(100, activation="relu"),
    Dense(100, activation="relu"),
    Dense(10, activation="sigmoid")
])

# Compile the model
model.compile(optimizer="adam",
              loss="sparse_categorical_crossentropy",
              metrics=["sparse_categorical_accuracy"])

# Train the model
history = model.fit(X_train, y_train,
                    batch_size=32, epochs=50,
                    validation_data=(X_test, y_test), verbose=0)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

# Plot the validation accuracy over epochs
plt.plot(history.history['val_sparse_categorical_accuracy'])
plt.xlabel('Epochs')
plt.ylabel('Validation Accuracy')
plt.show()




[2484 3621 7926 ... 5541 2568 7662]


NameError: name 'Sequential' is not defined

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(16,)),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
#Tensorboard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
model.fit(x=X_train, y=y_train, epochs=6,callbacks=[tensorboard_callback])

### Evaluate & Export

In [ ]:
loss, acc = model.evaluate(X_test, y_test)
# model.summary()
# model.save("model2")

In [ ]:
### CONVERT TO TFLITE
converter = tf.lite.TFLiteConverter.from_keras_model(model)
model2_tflite = converter.convert()
with open('model2.tflite', 'wb') as f: f.write(model2_tflite)

In [ ]:
### QUANTIZE
def representative_dataset():
  for d in inputs:
    # d = np.expand_dims(d, axis=0)
    yield [tf.dtypes.cast(d, tf.float32)]

# print(dataset.cardinality().numpy())
# print(tf.shape(dataset))
# model.summary()

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8
tflite_quant_model2 = converter.convert()
with open('model2_quant', 'wb') as f: f.write(tflite_quant_model2)

# Predict

## Reformat Input

In [ ]:
def resize_video(input_path, output_path, width, height):
  # Open the video file
  video = cv2.VideoCapture(input_path)

  # Get the original video's width and height
  original_width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
  original_height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
  
  # Create a VideoWriter object to save the resized video
  fourcc = cv2.VideoWriter_fourcc(*'mp4v') # Codec for the output video
  fps = video.get(cv2.CAP_PROP_FPS)
  writer = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
  
  while True:
    # Read a frame from the original video
    ret, frame = video.read()
    if not ret: break
    # Resize the frame to the desired width and height
    resized_frame = cv2.resize(frame, (width, height))
    # Write the resized frame to the output video file
    writer.write(resized_frame)

# Release the video capture and writer objects
  video.release()
  writer.release()

In [ ]:
# Resize vid
video_path = "compilation_cut.mp4"
output_path = "compilation_cut_resized.mp4"
target_width = 224
target_height = 224

#resize_video(video_path, output_path, target_width, target_height)

# Load resized vid
cap = cv2.VideoCapture(output_path)
# frame_rate = 30
# cap.set(cv2.CAP_PROP_FPS, frame_rate) DOESN'T WORK
print(cap.get(cv2.CAP_PROP_FPS))

## Pass into model 1

In [ ]:
# Initialize list to store the frame classifications
frame_classifications = []
moving_prob = []
# Loop through the frames of the video (need to change to 30 fps)
while True:
    ret, frame = cap.read() 
    #just need to figure out if this is 30 fps

    if not ret: # Break the loop if the video has ended
        break

    ''' Formulate Input Data (frame_rgb) '''
    # Convert the frame to RGB format
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    # Make frame input data and ensure its type matches the model
    frame_rgb = np.expand_dims(frame_rgb, axis=0)

    ''' Classify the Frame '''
    interpreter = tf.lite.Interpreter(model_path="model.tflite")
    interpreter.allocate_tensors()

    # get_output_details() and get_input_details() return list of dictionaries of tensor details
    # keys: name, index, shape, shape_signature, dtype, quantization, ...
    # len(input) = len(output) = 1, so access the first element
    output = interpreter.get_output_details()
    input = interpreter.get_input_details()
    output_index = output[0]['index']
    input_index = input[0]['index']

    # set input -> invoke -> access output
    interpreter.set_tensor(input_index, frame_rgb)
    interpreter.invoke()

    output_data = interpreter.get_tensor(output_index)
    # If the output_data shape is (batch_size, num_classes), select the first frame
    output_data = output_data[0]

    # Convert each entry into probability
    output_probs = tf.nn.softmax(output_data.astype(float))

    # Find the index of the highest probability
    predicted_index = np.argmax(output_data)

    # Assuming you have a list of class labels corresponding to the model's output classes
    class_labels = ["Moving", "Still"]

    # Get the predicted class label
    predicted_class = class_labels[predicted_index]

    # Print the predicted class label
    # print("Predicted Class:", predicted_class)
    frame_classifications.append((predicted_class, max(output_probs.numpy())))
    
    prob = np.around(max(output_probs.numpy()), decimals = 2)
    if predicted_class == "Still":
        
        moving_prob.append(np.subtract(1, prob))
    else:
        moving_prob.append(prob)

## Prepare data for model 2

In [ ]:
moving_probs_trimmed = moving_prob[:-(len(moving_prob)%16)]
model2_in = np.array(moving_probs_trimmed).reshape((len(moving_prob)//16, 16))
print(model2_in)

## Pass into Model 2 and get final prediction

In [ ]:
model2 = tf.keras.models.load_model("model2")

In [ ]:
vid_preds = model2.predict(model2_in)

In [ ]:
threshold = 0.88
bools = vid_preds.reshape((1, len(vid_preds))) > threshold
print(bools)
print(vid_preds.reshape((1, len(vid_preds))))